# The FrameworkClient

The FrameworkClient is a client side library to allow easy interaction with the services within CAVE (connectome annotation versioning engine, also known as Dynamic Annotation Framework), eg. the annotations, stateserver. The github repository is public:
https://github.com/seung-lab/AnnotationFrameworkClient

The library can be installed directly from the github repository or from the prebuilt versions using pip:
```
pip install annotationframeworkclient
```


## Tutorials

This tutorial mainly covers the interactions with the materialized annotation tables. More information and better explanations of the other functionalities of the client can be found in the following tutorial. Please be advised that depending on your permission level you may not be able to execute all queries in this tutorial with the preset parameters as it was written with defaults for iarpa's microns project:
https://github.com/seung-lab/AnnotationFrameworkClient/blob/master/FrameworkClientExamples.ipynb


## Authentication & Authorization

If this is your first time to interact with any part of CAVE, chances are you need to setup your local credentials for your FlyWire account first. Please follow the section "Setting up your credentials" at the beginning of the tutorial above to do so.

You will need to have access to the FlyWire's production dataset to retrieve annotations. Otherwise you will see

```HTTPError: 403 Client Error: FORBIDDEN for url```

errors upon querying the materialization server.

## Initialize FrameworkClient

The FrameworkClient is instantiated with a datastack name. A datastack is a set of segmentation, and annotation tables and lives within an aligned volume (the coordinate space). FlyWire's main datastack is `flywire_fafb_production`, the aligned volume is `fafb_seung_alignment_v0` (v14.1). For convenience, there are other defaults set on the datastack level.

In [1]:
import numpy as np
from annotationframeworkclient import FrameworkClient

In [2]:
datastack_name = "flywire_fafb_production"
client = FrameworkClient(datastack_name)

## Annotation tables

Annotations are represented by points in space and parameters (such as size, type). At specific timepoints, annotations are combined with the (proofread) segmentation to create a materialized version of the annotation table. The AnnotationEngine (`client.annotation`) owns the raw annotations and the Materialization Service (`client.materialize`) owns the materialized versions of these tables. 

To check what annotation tables are visible to you run

In [3]:
client.annotation.get_tables()

https://prod.flywire-daf.com/annotation/api/v2/aligned_volume/fafb_seung_alignment_v0/table


['synapses_nt_v1']

Every table has metadata associated with it which includes information about the owner/creator, a description and a schema that annotations in this table follow. Please review the metadata of any table you might use in the future before using it as it might contain instructions and restrictions for its usage and how to credit it's creators. For instance, the (v1) synapse table (`synapses_nt_v1`) includes an extensive description on all its columns, credits people that created it, contains instructions for citing this resource among others:

In [4]:
meta_data = client.annotation.get_table_metadata("synapses_nt_v1")
print(meta_data["description"])

FlyWire synapse description
Synapse version: 20191211
NT version: 20201223

Synapses in this table consist of a pre- and a postsynaptic point (in nm), confidence scores, and neurotransmitter information. 
The synapses were predicted by Buhmann et al [1] for the v14 alignment of the FAFB dataset. The FlyWire team remapped these synapses into the v14.1 space used by FlyWire with an accuracy of <64nm (therefore, this is a potential source of error). This version of the Buhmann et al. synapses was trained on the initial training set from the calyx and performance varies across brain areas accordingly.
Buhmann et al. assigned two scores to their synapses representing different measurements of confidence. The “connection_score” column contains the scores assigned by them during prediction (higher is more confident) and “cleft_score” contains the scores acquired by Buhmann et al. by using the synapse segmentation from Heinrich et al. [2] (higher is more confident). For the latter, Buhmann et 

The meta data contains information about the schema which ultimately determines how annotations in a table are structured. All annotations in a table follow the same schema. The synapse table follows the `fly_nt_synapse` schema:

In [5]:
meta_data["schema_type"]

'fly_nt_synapse'

## Materialized annotation tables & Queries

```
materialization = annotation + segmentation snapshot
```

As the segmentation and annotations change over time, we need to create snapshots of a combined view of them (materialized versions). Materialized versions of the annotation tables are (automatically) generated at a certain frequency. In addition to that, we are planning to include an option to retrieve any timestamp since the latest materialization ("live") but that is not available at the moment. 

There are usually a number of materialized versions available at the same time:

In [6]:
client.materialize.get_versions()

[15, 16, 17, 18, 19, 20, 22, 26, 27]

Each version comes with meta data about the time when it was created and when it will be deleted (expired). Different tables have different lifetimes and some may be LTS versions. The exact frequency and lifetime of tables will depend on how the community is using these tables. 

In [10]:
client.materialize.get_version_metadata(27)

{'time_stamp': '2021-04-19T08:10:00.253566',
 'id': 15,
 'datastack': 'flywire_fafb_production',
 'valid': True,
 'expires_on': '2021-04-26T08:10:00.253566',
 'version': 27}

Each materialization version contains a set of annotation tables. At the moment all tables are included in a materialization but in the future we might not include all tables in every materialization:

In [12]:
client.materialize.get_tables(version=27)

['synapses_nt_v1']

### Queries

Here, we demonstrate some queries with the synapses from Buhmann et al..

Each table in this list is stored as a SQL table on the backend. The client allows users to query these tables through the frontend of the Materialization Service conventiently without the need for SQL specific language. The client will format the results as pandas dataframes. Queries are restricted to a size of 200k rows to not overwhelm the server. Should a query result in a larger list of rows, only the first 200k are returned. For bulk downloads (eg. for data preservation before a publication) please contact us.

To deomstrate this this query would pull the entire table but will only gather 200k rows (should take <2min). In the future, a warning will be raised in such cases but at the moment, the server silently cuts the query short.

In [14]:
%%time

syn_df = client.materialize.query_table("synapses_nt_v1", materialization_version=27)
len(syn_df)

Wall time: 13 s


200000

Here, we set the materialization version specifically. If the materialization version is not specified, the query defaults to the most recent version.

Let's take a brief look at the columns to illustrate how the materialization extends an annotation table:

In [15]:
syn_df.head()

,id,valid,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,connection_score,cleft_score,gaba,ach,glut,oct,ser,da,valid_nt,pre_pt_position,post_pt_position
0,0,True,76916748439627068,720575940620535651,76916748439628340,720575940619047152,7.635252,0.0,0.248573,0.072179,0.057031,0.019089,0.112843,0.490285,True,"[366196, 244760, 73920]","[366296, 244740, 73960]"
1,1,True,76916748439635341,720575940620535651,76916748439614229,720575940616174657,15.259161,0.0,0.169801,0.312128,0.068937,0.044355,0.268214,0.136565,True,"[364864, 245640, 73960]","[364708, 245588, 73960]"
2,2,True,76916748439614229,720575940616174657,76916748439623112,720575940545890960,10.514650,0.0,0.616433,0.223582,0.047966,0.052839,0.028816,0.030364,True,"[364564, 244904, 73960]","[364680, 244840, 74000]"
3,3,True,76916748439632311,720575940620535651,76916748439633329,720575940627209594,71.585907,0.0,0.486084,0.209755,0.246938,0.000980,0.027569,0.028674,True,"[364600, 244468, 74200]","[364552, 244344, 74200]"
4,4,True,76916748439652476,720575940610019251,76916748439632348,720575940620535651,21.044254,0.0,0.338186,0.584693,0.027468,0.000612,0.023012,0.026030,True,"[364952, 244764, 74240]","[364860, 244796, 74200]"


Annotations consist of parameters and spatial points. Some or all of these spatial points are what we call "BoundSpatialPoints". These are linked to the segmentation during materialization. The synapse tables have two such points (`pre_pt`, `post_pt`). Per point there are three columns: `*_position`, `*_supervoxel_id`, `*_root_id`. Supervoxels are the small atomic segments, and root ids describe large components (neurons) consisting of many supervoxels. A root id always refers to the same version of a neuron and represents a snapshot in time in its own right. For a given annotation id (`id`), all but the `*_root_id` columns stay constant between materializations. 

`query_table` has three parameters to define filters: filter_in_dict, filter_out_dict, filter_equal_dict. More options will be added. This can be used to query synapses between any lists of neurons. For instance, to query the outgoing synapses of an AMMC-B1 neuron we included in the FlyWire paper:
(see the next section for how to come up with a specific root id)

In [17]:
%%time

syn_df = client.materialize.query_table("synapses_nt_v1", materialization_version=15,
                                        filter_in_dict={"pre_pt_root_id": [720575940627197566]})

CPU times: user 79.2 ms, sys: 12.6 ms, total: 91.8 ms
Wall time: 6.22 s


As described in the metadata above, we suggest filtering the synapse table using the `cleft_score` and `connection_score`. Tuning these will help to reduce the number of false positive synapses in the list. The best threshold(s) will depend on the specific neurons included in the analysis. Here we will just remove all synapses with a `cleft_score < 50`.

In [16]:
syn_df = syn_df[syn_df["cleft_score"] >= 50]

Some postsynaptic partners have a 0 id. Many of these are due to the synapse prediction covering a bigger space than the segmentation. Here, we remove these along with synapses onto itself as we are confident that this cell does not make autapses.

In [17]:
syn_df = syn_df[syn_df["pre_pt_root_id"] != syn_df["post_pt_root_id"]]
syn_df = syn_df[syn_df["post_pt_root_id"] != 0]

This synapse table comes with neurotransmitter prediction from the work of Eckstein et al.. Please review the description in the metadata to understand the caveats of this data with regards your analysis. Here, we just look at the mean of the probablities of all outgoing synapses which shows that this neuron's neurotransmitter is very likely acetylcholine.

In [18]:
np.mean(syn_df[["gaba", "ach", "glut", "oct", "ser", "da"]])

gaba    0.185471
ach     0.459503
glut    0.203313
oct     0.006671
ser     0.043023
da      0.102020
dtype: float64

Here we take a brief look at the postsynaptic partners and sorting them by number of synapses; displaying the top 10:

In [19]:
u_post_root_ids, c_post_root_ids = np.unique(syn_df["post_pt_root_id"], return_counts=True)

sorting = np.argsort(c_post_root_ids)[::-1][:10]
list(zip(u_post_root_ids[sorting], c_post_root_ids[sorting]))

[(720575940630739501, 266),
 (720575940639616987, 206),
 (720575940628312684, 173),
 (720575940608155950, 172),
 (720575940614678587, 162),
 (720575940607152067, 159),
 (720575940610093810, 158),
 (720575940622144776, 158),
 (720575940645191812, 148),
 (720575940631102675, 138)]

The main target is an AMMC-A1 (720575940613535430) which is a connection we described in Figure 6 in the FlyWire paper.

We can further restrict the query by filtering the postsynaptic targets. For instance this query will only return the synapses between the these two root ids.

In [20]:
syn_df = client.materialize.query_table("synapses_nt_v1", materialization_version=15,
                                        filter_in_dict={"pre_pt_root_id": [720575940627197566],
                                                        "post_pt_root_id": [720575940613535430]})
syn_df = syn_df[syn_df["cleft_score"] >= 50]

syn_df

,id,valid,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,connection_score,cleft_score,gaba,ach,glut,oct,ser,da,valid_nt,pre_pt_position,post_pt_position
0,6162631,True,77761104918444158,720575940627197566,77761104918444188,720575940613535430,326.542755,102.0,0.147903,0.462820,0.375533,0.005401,0.001151,0.007191,True,"[414712, 243344, 145120]","[414600, 243276, 145120]"
1,6180240,True,77761173637894262,720575940627197566,77761173637888123,720575940613535430,227.389908,158.0,0.011852,0.966436,0.005491,0.002103,0.000580,0.013538,True,"[414292, 245008, 144520]","[414160, 245016, 144520]"
3,8180655,True,77761173637907153,720575940627197566,77761173637915205,720575940613535430,49.321461,176.0,0.016921,0.900563,0.015684,0.001348,0.005939,0.059545,True,"[415680, 247148, 144920]","[415632, 247108, 145040]"
4,40964236,True,77691492021634849,720575940627197566,77691492021615097,720575940613535430,7.644144,62.0,0.024671,0.912242,0.009162,0.000161,0.018725,0.035040,True,"[412888, 286088, 130840]","[412960, 286024, 130840]"
5,51374115,True,77691285729051546,720575940627197566,77691285729087258,720575940613535430,167.695801,106.0,0.025043,0.350947,0.025653,0.002037,0.509739,0.086582,True,"[410048, 275336, 92480]","[410000, 275244, 92520]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,236982393,True,77761585821126507,720575940627197566,77761585821131453,720575940613535430,476.344971,145.0,0.009036,0.962195,0.000537,0.000177,0.001508,0.026548,True,"[415872, 269404, 120080]","[415864, 269284, 120040]"
122,237434920,True,77832023284777092,720575940627197566,77832023284775770,720575940613535430,58.748398,144.0,0.032626,0.951942,0.001326,0.000276,0.000651,0.013178,True,"[418172, 273748, 119600]","[418084, 273656, 119560]"
123,237434941,True,77831954565287519,720575940627197566,77831954565292946,720575940613535430,37.415009,145.0,0.013553,0.776630,0.010634,0.002239,0.005292,0.191653,True,"[417720, 272816, 119840]","[417708, 272892, 119880]"
124,237434942,True,77832023284765928,720575940627197566,77832023284771485,720575940613535430,161.775467,141.0,0.023719,0.890757,0.013778,0.002228,0.019519,0.049999,True,"[417900, 272952, 119880]","[417844, 273044, 119880]"


## Retrieving matching root ids

Neuroglancer shows the most recent version of the segmentation by default. Neurons that have been updated since a materialized version are not included in a table of that version. To reconcile this, users need to look up root ids for their data with a timestamp. 

We generally recommend storing annotations as points in space as these can be mapped to root ids easily (that's basically what materialization is). Soon, users will be able to create their own annotation tables and CAVE will provide fitting root ids automatically. Still, use cases will arrive that require a mnual materialization by the user:


### Programmatically

The client has an interface to the chunkedgraph (see Section 5 in [the related tutorial](https://github.com/seung-lab/AnnotationFrameworkClient/blob/master/FrameworkClientExamples.ipynb) which allows users to query a root id for a given supervoxel id. Supervoxel ids can be retrieved from the segmentation using [cloudvolume](https://github.com/seung-lab/cloud-volume/).


### Neuroglancer

The segmentation layer has an option under the tab "graph" to lock a layer to a specific timestamps. Then, root ids are looked up with this specific timestamp (proofreading is not possible in this mode). Be aware that this mode does not prevent the pasting of root ids from different timestamps into the layer as that circumvents the lookup to the server.

### Timestamps

Timestamps are _always_ UTC. 

Please be aware that the package or browser you are using might format timestamps in your local timezone. The timestamp for all annotation tables within a materialization are the same:

In [21]:
client.materialize.get_version_metadata(27)

{'datastack': 'flywire_fafb_production',
 'version': 27,
 'expires_on': '2021-04-26T08:10:00.253566',
 'id': 15,
 'valid': True,
 'time_stamp': '2021-04-19T08:10:00.253566'}

## Creating neuroglancer links programmatically

We are building infrastructure into neuroglancer to display this information there while browsing neurons. Until this is ready, the most convenient way to visualize this information in neuroglancer is to programmatically create neuroglancer state and to upload them to the state server. The links can then be distributed. 

[NeuroglancerAnnotationUI (nglui)](https://github.com/seung-lab/NeuroglancerAnnotationUI)  makes programmatic creation of neuroglancer states convenient. The [statebuilder examples](https://github.com/seung-lab/NeuroglancerAnnotationUI/blob/master/examples/statebuilder_examples.ipynb) shows how one can directly from dataframes as the one above to neuroglancer states. The [related tutorial on this client](https://github.com/seung-lab/AnnotationFrameworkClient/blob/master/FrameworkClientExamples.ipynb) shows under "4. JSON Service" how this client can be used to upload states to the server and to create neuroglancer links.


## Further references


More examples for the usage of CAVE can be found in a related project:

https://github.com/AllenInstitute/MicronsBinder

A rough overview of the structure of our backend services can be found here:

https://github.com/seung-lab/AnnotationPipelineOverview

## Credit

CAVE is developed at Princeton University and the Allen Institute for Brain Science within the iarpa MICrONs project and the FlyWire project. Main contributors to the design and backend development 
are Derrick Brittain, Forrest Collman, Sven Dorkenwald, Chris Jordan, Casey Schneider-Mizell

A citable publication is in the works. Please contact us if you are interested in using CAVE on another dataset. 